In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from deap import base, creator, tools, algorithms  # Assuming DEAP library for GEP
import numpy as np
import matplotlib.pyplot as plt

# Dataset paths
train_data_dir = 'C:/Users/YELLEPEDDI KRISHNA/Desktop/Assish1/train'
test_data_dir = 'C:/Users/YELLEPEDDI KRISHNA/Desktop/Assish1/Test'
valid_data_dir = 'C:/Users/YELLEPEDDI KRISHNA/Desktop/Assish1/val'

# Image Data Generators for train, validation, and test
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(380, 380), batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_data_dir, target_size=(380, 380), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(380, 380), batch_size=32, class_mode='categorical')

# Load EfficientNet B4 Model
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380, 380, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Dense layer
predictions = Dense(4, activation='softmax')(x)  # 4 classes

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Feature maps visualization from 1st Convolutional Layer
layer_outputs = [layer.output for layer in base_model.layers if 'conv' in layer.name]
conv_model = Model(inputs=base_model.input, outputs=layer_outputs[0])  # First convolutional layer

def visualize_feature_maps(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(380, 380))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    feature_maps = conv_model.predict(img_array)

    fig, axes = plt.subplots(8, 8, figsize=(12, 12))  # Displaying 64 feature maps
    axes = axes.flatten()
    for i in range(64):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
    plt.show()

# Dense layer feature maps visualization
dense_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)

def visualize_dense_feature_maps(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(380, 380))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    dense_feature_maps = dense_layer_model.predict(img_array)
    plt.figure(figsize=(10, 5))
    plt.plot(dense_feature_maps[0])
    plt.title("Dense Layer Feature Map")
    plt.show()

# GEP Algorithm setup (assuming DEAP library)
def GEP():
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()

    # Define your hyperparameters for optimization
    toolbox.register("learning_rate", np.random.uniform, 0.0001, 0.01)
    toolbox.register("dropout_rate", np.random.uniform, 0.2, 0.5)
    toolbox.register("batch_size", np.random.randint, 16, 64)

    # Use the hyperparameters to create individuals
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                     (toolbox.learning_rate, toolbox.dropout_rate, toolbox.batch_size), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Define fitness function (train the model and return accuracy)
    def evaluate(individual):
        lr, dropout, batch_size = individual[0], individual[1], int(individual[2])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                      loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_generator, validation_data=valid_generator, epochs=5, batch_size=batch_size)
        _, accuracy = model.evaluate(valid_generator)
        return accuracy,

    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Evolve population
    population = toolbox.population(n=10)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=True)

# Train and visualize feature maps
history = model.fit(train_generator, validation_data=valid_generator, epochs=10)
visualize_feature_maps('C:/Users/YELLEPEDDI KRISHNA/Desktop/CT-Images/A.png')
visualize_dense_feature_maps('C:/Users/YELLEPEDDI KRISHNA/Desktop/CT-Images/A.png')

# Plot training accuracy and loss
def plot_training_history(history):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(False)  # As per your preference to remove grid
    plt.show()

plot_training_history(history)
